In [1]:
#pip install openprompt

In [21]:
import csv
csv.field_size_limit(1410720)

# 初始化字典
trec_dict = {}
q_dict = {}
query_id_counts = {}


# 从trec.txt中读取数据
with open('401-450.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        query_content = row[1]  # 第二列是查询内容
        trec_dict[query_id] = query_content

# 从lmdir.txt中读取数据
with open('401-450diri1000.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        doc_content = row[6]  # 第7列是文档内容
        # 计算当前查询id的文档数量
        if query_id_counts.get(query_id, 0) >= 5:
            continue
        
        
        # 在文档内容前添加标识
        # 将查询id和带标识的文档内容添加到字典中
        key = query_id + '\t' + trec_dict.get(query_id, '')
        i = len(q_dict.get(key, []))+1
        doc_content = f'Document {i} : {doc_content}'
        
        if key in q_dict:
            q_dict[key].append(doc_content)
        else:
            q_dict[key] = []
            q_dict[key].append(doc_content)
            
        query_id_counts[query_id] = query_id_counts.get(query_id, 0) + 1



# 打印出第一个查询id及其对应的文档内容，以验证代码是否正确
for query_id in list(q_dict.keys())[:1]:
    print("Query :", query_id)
    print("Document Contents:", q_dict[query_id])


Query : 401	What language and cultural differences impede the integration of foreign minorities in Germany?
Document Contents: ['Document 1 : FBIS4-18372 "drweu064_h_94001" FBIS-WEU-94-064 Daily Report 3 Apr 1994 GERMANY EU Minority Blocking Vote `Overestimated\' EU Minority Blocking Vote `Overestimated\' LD0304093094 Berlin DDP/ADN in German 0415 GMT 3 Apr 94 LD0304093094 Berlin DDP/ADN German BFN [Text] Nuremberg (DDP/ADN) -- In the view of Foreign Minister Klaus Kinkel, the practical significance of the minority blocking vote in the EU [European Union] Council of Ministers, which has until recently been strongly disputed because of the membership negotiations of the EFTA [European Free Trade Association] states, is being overestimated. In any case, in the EU the principle of unanimity governed complex matters, and the qualified majority vote was decisive for issues concerning the internal market, Kinkel told the DDP/ADN news agency in Nuremberg. Moreover, the EU foreign ministers ha

In [22]:
# q_dict['264014	how long is life cycle of flea']
type(q_dict)

dict

In [23]:
# from langchain.llms import OpenAI
# from langchain.chains.summarize import load_summarize_chain
# from langchain.chains import AnalyzeDocumentChain

# llm = OpenAI(openai_api_key="",temperature=0)


# def summarize_doc(doc):
#     # 处理文档的代码在这里
#     chain = load_summarize_chain(llm, chain_type="map_reduce")

#     summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=chain)

#     processed_doc = summarize_document_chain.run(doc)
      
#     return processed_doc


# for key, value in q_dict.items():
#     new_value = []
#     for doc in value:
#         new_value.append(summarize_doc(doc))
#     q_dict[key] = new_value


In [24]:
q_dict['401	What language and cultural differences impede the integration of foreign minorities in Germany?']

['Document 1 : FBIS4-18372 "drweu064_h_94001" FBIS-WEU-94-064 Daily Report 3 Apr 1994 GERMANY EU Minority Blocking Vote `Overestimated\' EU Minority Blocking Vote `Overestimated\' LD0304093094 Berlin DDP/ADN in German 0415 GMT 3 Apr 94 LD0304093094 Berlin DDP/ADN German BFN [Text] Nuremberg (DDP/ADN) -- In the view of Foreign Minister Klaus Kinkel, the practical significance of the minority blocking vote in the EU [European Union] Council of Ministers, which has until recently been strongly disputed because of the membership negotiations of the EFTA [European Free Trade Association] states, is being overestimated. In any case, in the EU the principle of unanimity governed complex matters, and the qualified majority vote was decisive for issues concerning the internal market, Kinkel told the DDP/ADN news agency in Nuremberg. Moreover, the EU foreign ministers had undertaken to reexamine the whole procedure, which has now been decided, in 1996. It had at any rate been right not to let th

In [8]:
import json

def save_dict_to_file(q_dict, filename):
    with open(filename, 'w') as f:
        json.dump(q_dict, f)

save_dict_to_file(q_dict, 'q_dict401-450.txt')


In [1]:
import json

def load_dict_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# 使用示例
q_dict = load_dict_from_file('q_dict401-450.txt')
#print(q_dict['264014	how long is life cycle of flea'])


In [2]:
q_dict['401	What language and cultural differences impede the integration of foreign minorities in Germany?']

[' Foreign Minister Klaus Kinkel believes the minority blocking vote in the European Union Council of Ministers is being overestimated and that the principle of unanimity and qualified majority vote are more important. He also rejects criticism from the CSU-led Bavarian Government that German interests are not being represented. He believes Germany is being viewed critically due to its past, but there is no reason to be overly cautious.',
 ' Ilan Mor, spokesman of the Israeli Embassy in Bonn, expresses his concern over the attack on a synagogue in Luebeck, Germany and believes that anti-Semitism exists in the country. He mentions an opinion poll which showed that 22% of people do not want a Jewish neighbor and emphasizes that Israel is closely monitoring events in Germany due to the events of 1933 and 1945.',
 ' German Chancellor Helmut Kohl is in a confrontation with Turkey over arms shipments, which must be used for defense within the NATO alliance. Hans-Ulrich Klose criticized Kohl 

In [3]:
import itertools
from openprompt.data_utils import InputExample

input_examples = []

# 使用itertools.combinations来获取所有的两两组合，不会重复
for query_id_content in q_dict.keys():
    # 分割查询ID和查询内容
    query_id, query = query_id_content.split('\t', 1)
    documents = q_dict[query_id_content]
    
    # 构建InputExample
    input_example = InputExample(
        guid= len(input_examples),
        label= None,
        meta= {
            "queryid": query_id,
            "query": query,
            "documents": documents,
            #"querybid": queryb_id,
            #"queryb": queryb,
            #"documentsb": documentsb
        },
        text_a= "",
        text_b= "",
        tgt_text= query_id
    )
    
    # 将InputExample添加到列表中
    input_examples.append(input_example)


D:\pd\Anaconda3\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [4]:
print(input_examples[2])

{
  "guid": 2,
  "label": null,
  "meta": {
    "documents": [
      " A new study has found that the drug etidronate, used to treat Paget's disease and a bone disorder in cancer patients, can halve the rate of vertebral fractures in women with osteoporosis. The drug is sold under the brand name Didronel and the manufacturer is applying to the FDA for approval to use the drug to treat osteoporosis. The study also found that the drug increased spinal bone density by an average of 4-5%. Further studies are needed to explore the drug's long-term effects and to compare its effectiveness to other osteoporosis treatments.",
      " Australian researchers have found that daughters of women with osteoporosis are more likely to develop the disease than those whose mothers do not have it, suggesting that genetics may be a factor in its development. The study compared 32 women with mothers who had osteoporosis to 22 women with mothers who did not, and found that the former had thinner bones.",
  

In [5]:
for example in input_examples:
    example.meta['documents'] = example.meta['documents'][0:2]

In [6]:
for example in input_examples:
    # 对'documentsa'列表中的文档编上号并拼接
    documents = example.meta['documents']
    documents = [f'Document {i+1}: {doc}' for i, doc in enumerate(documents)]
    example.meta['documents'] = ' '.join(documents)

In [7]:
input_examples[2]

{
  "guid": 2,
  "label": null,
  "meta": {
    "documents": "Document 1:  A new study has found that the drug etidronate, used to treat Paget's disease and a bone disorder in cancer patients, can halve the rate of vertebral fractures in women with osteoporosis. The drug is sold under the brand name Didronel and the manufacturer is applying to the FDA for approval to use the drug to treat osteoporosis. The study also found that the drug increased spinal bone density by an average of 4-5%. Further studies are needed to explore the drug's long-term effects and to compare its effectiveness to other osteoporosis treatments. Document 2:  Australian researchers have found that daughters of women with osteoporosis are more likely to develop the disease than those whose mothers do not have it, suggesting that genetics may be a factor in its development. The study compared 32 women with mothers who had osteoporosis to 22 women with mothers who did not, and found that the former had thinner bone

In [8]:
# 假设 input_examples 是你的数据集
text_list = []
for example in input_examples:
    #text = 'Given the following two queries: the first query is \"' + example.meta['querya'] + '\", and the second query is \"' + example.meta['queryb'] + '\". Compared to the first query, the second query is more {"mask"}.'
    #text = 'Given the following two queries and their respective top one document----the first query is \"' + example.meta['querya'] + '\" with its top one search result as \"' + example.meta['documentsa'] + '\", and the second query is \"' + example.meta['queryb'] + '\" with its top one search result as \"' + example.meta['documentsb'] + '\". Compared to the first query, the second query is more {"mask"}.'
    #text = 'Query \"'+  example.meta['query'] + '\" is {"mask"}.' 
    text = 'Given the query \"'+  example.meta['query'] + '\" and its top two search results \"'+  example.meta['documents'] + '\" , it can be inferred that the query is {"mask"}.'
    text_list.append(text)


In [9]:
print(text_list[0])
print(text_list[1])
print(text_list[2])

Given the query "What language and cultural differences impede the integration of foreign minorities in Germany?" and its top two search results "Document 1:  Foreign Minister Klaus Kinkel believes the minority blocking vote in the European Union Council of Ministers is being overestimated and that the principle of unanimity and qualified majority vote are more important. He also rejects criticism from the CSU-led Bavarian Government that German interests are not being represented. He believes Germany is being viewed critically due to its past, but there is no reason to be overly cautious. Document 2:  Ilan Mor, spokesman of the Israeli Embassy in Bonn, expresses his concern over the attack on a synagogue in Luebeck, Germany and believes that anti-Semitism exists in the country. He mentions an opinion poll which showed that 22% of people do not want a Jewish neighbor and emphasizes that Israel is closely monitoring events in Germany due to the events of 1933 and 1945." , it can be infe

In [10]:
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate
from openprompt.prompts import ManualVerbalizer
from openprompt import PromptForClassification
from openprompt import PromptDataLoader
from openprompt.plms.utils import TokenizerWrapper
from collections import defaultdict
import torch

classes = [ 
    "specific",
    "generic"
]
scores = defaultdict(float)

#plm, tokenizer, model_config, WrapperClass = load_plm("t5","google/flan-t5-base")
#plm, tokenizer, model_config, WrapperClass = load_plm("albert", "albert-base-v2")
#plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-base")
#plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "gpt2-large")
plm, tokenizer, model_config, WrapperClass = load_plm("opt", "facebook/opt-iml-max-1.3b")

x=0
for t in text_list:
    print("---------------------------------------------------------------")
    #print(t)
    dataset = []
    dataset.append(input_examples[x])
    x = x + 1
    promptTemplate = ManualTemplate(
        text = t,
        tokenizer = tokenizer,
    )
    promptVerbalizer = ManualVerbalizer(
        classes = classes,
        label_words = {
            "specific":["specific","precise","exact","well-formed","clear"],
            "generic":["generic","general","common","universal","ambiguous", "ill-formed", "vague"]
        },
        tokenizer = tokenizer,
    )
    promptModel = PromptForClassification(
        template = promptTemplate,
        plm = plm,
        verbalizer = promptVerbalizer,
    )
    data_loader = PromptDataLoader(
        dataset = dataset,
        tokenizer = tokenizer,
        template = promptTemplate,
        tokenizer_wrapper_class=WrapperClass,
        #max_seq_length=512, decoder_max_length=3,batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False, truncate_method="tail"
    )
    for batch in data_loader:

        with torch.no_grad():
            logits = promptModel(batch)
            preds = torch.argmax(logits, dim = -1)
            probs = torch.softmax(logits, dim=-1)
          
        for i, prob in enumerate(probs):
            # 获取query的id
            query_id = batch.tgt_text[i]
            print(query_id)

            print(prob[classes.index('specific')],prob[classes.index('generic')])

            # 根据模型的预测概率给query加分
            scores[query_id] += prob[classes.index('specific')]
            
    


---------------------------------------------------------------


tokenizing: 1it [00:00, 199.97it/s]


401
tensor(0.2928) tensor(0.7072)
---------------------------------------------------------------


tokenizing: 1it [00:00, 154.30it/s]


402
tensor(0.3855) tensor(0.6145)
---------------------------------------------------------------


...

448
tensor(0.2136) tensor(0.7864)
---------------------------------------------------------------


tokenizing: 1it [00:00, 237.96it/s]


449
tensor(0.2610) tensor(0.7390)
---------------------------------------------------------------


tokenizing: 1it [00:00, 166.63it/s]


450
tensor(0.2856) tensor(0.7144)


In [11]:
scores

defaultdict(float,
            {'401': tensor(0.2928),
             '402': tensor(0.3855),
             '403': tensor(0.3495),
             '404': tensor(0.2646),
             '405': tensor(0.3649),
             '406': tensor(0.2410),
             '407': tensor(0.3315),
             '408': tensor(0.3539),
             '409': tensor(0.3089),
             '410': tensor(0.2783),
             '411': tensor(0.4552),
             '412': tensor(0.2348),
             '413': tensor(0.3188),
             '414': tensor(0.2682),
             '415': tensor(0.3293),
             '416': tensor(0.3188),
             '417': tensor(0.3351),
             '418': tensor(0.2338),
             '419': tensor(0.2612),
             '420': tensor(0.0653),
             '421': tensor(0.2153),
             '422': tensor(0.2711),
             '423': tensor(0.4094),
             '424': tensor(0.3049),
             '425': tensor(0.2174),
             '426': tensor(0.2388),
             '427': tensor(0.3273),
         

In [12]:
sorted_queries = sorted(scores.items(), key=lambda item: item[1], reverse=True)


In [13]:
sorted_queries 

[('411', tensor(0.4552)),
 ('439', tensor(0.4347)),
 ('431', tensor(0.4331)),
 ('423', tensor(0.4094)),
 ('443', tensor(0.3872)),
 ('402', tensor(0.3855)),
 ('429', tensor(0.3744)),
 ('405', tensor(0.3649)),
 ('408', tensor(0.3539)),
 ('403', tensor(0.3495)),
 ('442', tensor(0.3371)),
 ('444', tensor(0.3360)),
 ('417', tensor(0.3351)),
 ('447', tensor(0.3335)),
 ('407', tensor(0.3315)),
 ('430', tensor(0.3293)),
 ('415', tensor(0.3293)),
 ('427', tensor(0.3273)),
 ('434', tensor(0.3211)),
 ('416', tensor(0.3188)),
 ('413', tensor(0.3188)),
 ('409', tensor(0.3089)),
 ('424', tensor(0.3049)),
 ('401', tensor(0.2928)),
 ('450', tensor(0.2856)),
 ('435', tensor(0.2814)),
 ('410', tensor(0.2783)),
 ('437', tensor(0.2762)),
 ('422', tensor(0.2711)),
 ('414', tensor(0.2682)),
 ('404', tensor(0.2646)),
 ('419', tensor(0.2612)),
 ('449', tensor(0.2610)),
 ('438', tensor(0.2440)),
 ('406', tensor(0.2410)),
 ('440', tensor(0.2406)),
 ('426', tensor(0.2388)),
 ('412', tensor(0.2348)),
 ('418', ten

In [14]:
query_dict = dict(sorted_queries)

In [15]:
query_dict = {k: v.item() for k, v in query_dict.items()}

In [16]:
query_dict

{'411': 0.45518794655799866,
 '439': 0.4346924424171448,
 '431': 0.43310168385505676,
 '423': 0.4093930721282959,
 '443': 0.38718536496162415,
 '402': 0.38554543256759644,
 '429': 0.3743726909160614,
 '405': 0.3648919463157654,
 '408': 0.35385698080062866,
 '403': 0.3494800329208374,
 '442': 0.3370559513568878,
 '444': 0.3360496759414673,
 '417': 0.33513087034225464,
 '447': 0.33349451422691345,
 '407': 0.3314908742904663,
 '430': 0.3292964994907379,
 '415': 0.32928627729415894,
 '427': 0.327295184135437,
 '434': 0.3210892379283905,
 '416': 0.31877246499061584,
 '413': 0.3187713921070099,
 '409': 0.30894002318382263,
 '424': 0.30494073033332825,
 '401': 0.2928086519241333,
 '450': 0.28560590744018555,
 '435': 0.2813933789730072,
 '410': 0.278344064950943,
 '437': 0.27616482973098755,
 '422': 0.2710694372653961,
 '414': 0.2682437300682068,
 '404': 0.2645978629589081,
 '419': 0.26115986704826355,
 '449': 0.2610084116458893,
 '438': 0.24403885006904602,
 '406': 0.24096937477588654,
 '440'

In [17]:
with open('401-450gt.txt', 'r') as f:
    gt = dict(line.strip().split() for line in f)

with open('401-450nqc.txt', 'r') as f:
    nqc = dict(line.strip().split() for line in f)

query_scores = list(query_dict.values())
#gtscores = [float(gt[key]) for key in query_dict.keys()]
#nqcscores = [float(nqc[key]) for key in query_dict.keys()]


In [18]:
keys = query_dict.keys()
gtscores = [float(gt[key]) for key in keys]
nqcscores = [float(nqc[key]) for key in keys]


In [19]:
keys

dict_keys(['411', '439', '431', '423', '443', '402', '429', '405', '408', '403', '442', '444', '417', '447', '407', '430', '415', '427', '434', '416', '413', '409', '424', '401', '450', '435', '410', '437', '422', '414', '404', '419', '449', '438', '406', '440', '426', '412', '418', '446', '425', '421', '448', '441', '436', '432', '445', '433', '428', '420'])

In [20]:
with open('opt_point_post2.txt', 'w', encoding='utf-8') as f:
    for key in query_dict.keys():
        f.write(f'{key}\t{query_dict[key]}\t{nqc[key]}\t{gt[key]}\n')


In [21]:
nqcscores

[8.253385543823242,
 4.02252721786499,
 -6.610340118408203,
 31.011184692382812,
 0.7333466410636902,
 2.892881155014038,
 8.541165351867676,
 8.07620906829834,
 6.774903774261475,
 45.91432189941406,
 2.685452461242676,
 11.117552757263184,
 1.1843476295471191,
 4.2269487380981445,
 7.814405918121338,
 11.102095603942871,
 14.683598518371582,
 13.625859260559082,
 1.5539339780807495,
 -14.283669471740723,
 0.6664474010467529,
 9.889849662780762,
 1.8052459955215454,
 0.5529524683952332,
 13.707488059997559,
 1.9212779998779297,
 9.120701789855957,
 2.51373553276062,
 3.7995567321777344,
 -8.841760635375977,
 3.8539679050445557,
 3.981992721557617,
 7.3450188636779785,
 2.7874035835266113,
 8.005461692810059,
 1.0883139371871948,
 1.6846462488174438,
 -0.05628133937716484,
 2.5999250411987305,
 2.6592535972595215,
 -12.099804878234863,
 1.7216060161590576,
 0.9927824139595032,
 8.429100036621094,
 2.1707212924957275,
 3.6750802993774414,
 11.008926391601562,
 1.462353229522705,
 4.1800

In [24]:
from scipy.stats.stats import pearsonr
p1 = pearsonr(query_scores, gtscores)
p2 = pearsonr(nqcscores, gtscores)
print(p1)
print(p2)

(0.15923310010948583, 0.26936109852781503)
(0.36221728005952814, 0.009740338185571448)


In [25]:
from scipy.stats.stats import kendalltau
k1 = kendalltau(query_scores, gtscores)
k2 = kendalltau(nqcscores, gtscores)
print(k1)
print(k2)

KendalltauResult(correlation=0.1346938775510204, pvalue=0.16752460252559565)
KendalltauResult(correlation=0.23265306122448978, pvalue=0.017126325378114097)
